In [32]:
import numpy as np
import matplotlib.pyplot as plt
import hmmlearn.hmm as hmm

In [2]:
model = hmm.CategoricalHMM()
model.n_components  = 2     # two states: hot and cold days
model.startprob_    = [ 0.8,0.2 ]
model.transmat_     = [ [0.6,0.4] , [0.5,0.5] ]
model.emissionprob_ = [ [0.2,0.4,0.4] , [0.5,0.4,0.1] ]

In [19]:
# problem 1
print("Problem 1")

logprob = model.score([[2,0,2]])
print("")
print( f"probability of states [2,0,2] is {np.exp(logprob):0.5f} , compared to 0.028562 computed by hand" )

state_to_temp={0:'H', 1:'C'}


logprob,seq = model.decode([[2,0,2]])
print("")
print("given an observation of [2,0,2], ")
print("most likely state sequence is " , [state_to_temp[s] for s in seq])
print("with likelihood " + f"{np.exp(logprob):0.5f}" + ", compared to 0.0128 computed by hand")

Problem 1

probability of states [2,0,2] is 0.02856 , compared to 0.028562 computed by hand

given an observation of [2,0,2], 
most likely state sequence is  ['H', 'C', 'H']
with likelihood 0.01280, compared to 0.0128 computed by hand


In [ ]:
# problem 2

$$pr(day1H | 3,1) = \frac{pr(3,1|day1H)pr(day1H)}{pr(3,1)}$$

Each of these three terms can be computed using the rules of probability:

$$pr(3,1 | day1H) = 0.4\left[0.6\times0.2\ +\ 0.4\times0.5\right] = 0.128$$

$$pr(day1H) = 0.8$$

$$pr(3,1) = H3H1 + H3C1 + C3H1 + C3C1 = 0.8\times0.6\times0.4\times0.2\ +\ ...\ =\ 0.1094$$

Combining we get $$pr(day1 | 3,1) = \frac{0.128\ \times\ 0.8}{0.1094} = 0.936$$

In [23]:
# confirm that pr(day 1 is hot given observations of 3 and then 1 ice cream)
_,posteriors = model.score_samples([[2,0]])
print(f"{posteriors[0,0]:0.5f}")

0.93601


In [27]:
# problem 3
data = np.load('hw_09_data.npy')

In [20]:
for state_num in range(1,5):
    model = hmm.GaussianHMM()
    model.n_components = state_num
    model.n_iter = 500
    model.covariance_type = "full"
    model.fit(data)
    scores = model.score(data)
    print(state_num , f"{scores:0.3f}")

1 -2048.551
2 -1866.892
3 -2048.295
4 -1781.022


Best fit is 4 states, followed by 3, and then 2. The data was actually generated using three states. What happens if we try using even more states?

In [3]:
for state_num in range(5,11):
    model = hmm.GaussianHMM()
    model.n_components = state_num
    model.n_iter = 500
    model.covariance_type = "full"
    model.fit(data)
    scores = model.score(data)
    print(state_num , f"{scores:0.3f}")

5 -1860.487
6 -1781.266
7 -1796.862
8 -1785.940
9 -1772.730
10 -1794.585


The quality of fit goes up a bit and then down a bit. We have to be mindful not to overfit the model with too many states.

We can also print the state transmission matrix to work out whether all the states are actually being used. If we do this, we see that the two-state solution has the fewest unused states. Even a three state model has one state that is hardly ever used. It's important to look at this and not just rely on the fit/score values.

In [24]:
model = hmm.GaussianHMM()
model.n_components = 5
model.n_iter = 500
model.covariance_type = "full"
model.fit(data)

for row in model.transmat_:
    for r in row:
        print(f"{r:0.3f}    ",end="")
    print("")

0.795    0.076    0.023    0.094    0.012    
0.000    0.797    0.000    0.197    0.005    
0.001    0.002    0.001    0.014    0.982    
0.442    0.169    0.250    0.078    0.061    
0.091    0.101    0.124    0.000    0.683    
